###  Activity 1: All Possible Regressions

This is a quick assignment.  I would expect it to only take a couple of hours to complete. The start of this notebook develops the example.  You will then parallelize the exammple using `joblib`. 

Run this on a machine that has at least 4 cores. Typically this is your laptop. The ugrad machines are also OK. Google Colab is not adequate. Fill out the requested cells toward the bottom of the notebook.

**Due date**: Tuesday September 12, 2022, 9:00 pm EDT.

**Instructions for Submission**: Submit via Gradescope.

#### Preparing the Environment

You will need a couple of packages installed to make this work.  Stop your instance of jupyter lab and then run

```
conda install pandas pandoc
jupyter lab
```
At this point, you should be ready to go.

#### Example code

This is a Python reimplementation of the Section 3.4. in _Matloff_, Parallel Computing for Data Science. It is based on data from https://www.kaggle.com/divan0/multiple-linear-regression. The notebook asks the question what combination of variables best predict the price of a house.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

#importing dataset using panda
dataset = pd.read_csv('../data/kc_house_data.csv')
#to see what my dataset is comprised of
dataset.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180.0,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170.0,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770.0,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050.0,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680.0,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
#dropping the id and date column
dataset = dataset.drop(['id','date'], axis = 1)

# clean out NaN and inf values
dataset = dataset[~dataset.isin([np.nan, np.inf, -np.inf]).any(1)]

/tmp/ipykernel_46/3445377822.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  dataset = dataset[~dataset.isin([np.nan, np.inf, -np.inf]).any(1)]


Let's first do a simple regression. How does square footage predict price? 

In [3]:
from sklearn.linear_model import LinearRegression

X = np.array(dataset.sqft_living)
Y = np.array(dataset.price)

# shape X into matrix of a single column
X = X.reshape((X.shape[0],1))

model = LinearRegression()
model.fit(X,Y)
r_sq = model.score(X,Y)
print('coefficient of determination:', r_sq)

coefficient of determination: 0.4928817196006994


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

# scatter data
plt.plot(X,Y,'o')
# best fit line
y_pred = model.intercept_ + model.coef_ * X
plt.plot(X, y_pred, color='red', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

We see that there is a strong correlation between square footage and house price.  The coefficient of determination measures the strength of the correlation and varies between 0 (no correlation) and 1.0 (perfectly correlated).

### Multiple Linear Regression

Adding more variables often improves the score.

In [5]:
X = np.array(dataset[['sqft_living','condition','yr_built']])
Y = np.array(dataset.price)

# shape X into matrix of a single column
X = X.reshape((X.shape[0],3))

model = LinearRegression()
model.fit(X,Y)
r_sq = model.score(X,Y)
print('coefficient of determination:', r_sq)

coefficient of determination: 0.5249639858277426


but some variables are confounding

In [6]:
X = np.array(dataset[['zipcode','floors','waterfront']])
Y = np.array(dataset.price)

# shape X into matrix of a single column
X = X.reshape((X.shape[0],3))

model = LinearRegression()
model.fit(X,Y)
r_sq = model.score(X,Y)
print('coefficient of determination:', r_sq)

coefficient of determination: 0.13588749977708692


This leads to a first parallel program.  What are the right set of variables? A brute force approach called _All Possible Regressions_ examines all combinations. So, let's build a big matrix that and we will regress on subsets.  We will look at all combinations of 1, 2, or 3 variables.

In [7]:
from itertools import combinations, chain

Y = np.array(dataset.price)
X = np.array(dataset.drop(['price'], axis=1))

## Let's choose all combinations of 1, 2, and 3 columns.
col_idxs = np.array(range(X.shape[1]))
combos = list(chain(combinations(col_idxs, 1), combinations(col_idxs, 2), combinations(col_idxs, 3)))

In [8]:
model = LinearRegression()

# do in a for loop (dumbest way)
r_sq_best = 0.0
for combo in combos:
    Xp = X[:,combo]
    model = model.fit(Xp,Y)
    r_sq = model.score(Xp,Y)
    if r_sq > r_sq_best:
        r_sq_best = r_sq
        combo_best = combo
    
print(r_sq_best, combo_best)

0.6095149101819035 (2, 6, 14)


The outcome is kind of crazy. The fields are `sqft-living`, `view`, and `latitude`. Latitude is probably a somewhat accurate proxy for wealth in this area, e.g. N of town richer than south of town. But, this is the kind of outcome that would not translate to other regions, i.e. is likely specific to this data.  _Neat_.

Back to performance.

In [9]:
%%timeit
model = LinearRegression()

r_sq_best = 0.0
for combo in combos:
    Xp = X[:,combo]
    model = model.fit(Xp,Y)
    r_sq = model.score(Xp,Y)
    if r_sq > r_sq_best:
        r_sq_best = r_sq
        combo_best = combo

1.64 s ± 23.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Let's see if flattening the loop matters. Replace for loop with a list comprehension of all combination of variables.

In [10]:
def r_sq_regression (combo):
    Xp = X[:,combo]
    model = LinearRegression()
    model = model.fit(Xp,Y)
    return model.score(Xp,Y)

In [11]:
r_sq_list = [ (combo, r_sq_regression(combo)) for combo in combos ]
r_sq_arr = np.array(r_sq_list, dtype=object)
r_sq_idx = np.argmax(r_sq_arr[:,1])
print(r_sq_arr[r_sq_idx])

[(2, 6, 14) 0.6095149101819035]


In [12]:
%%timeit
r_sq_list = [ (combo, r_sq_regression(combo)) for combo in combos ]
r_sq_arr = np.array(r_sq_list, dtype=object)
r_sq_idx = np.argmax(r_sq_arr[:,1])

1.62 s ± 21.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It didn't seem to help. But, this is a step toward parallelization.

### Exercise (This is the assignment)

1. Use `joblib.Parallel` and `joblib.delayed` to parallelize the computation of the calls to `r_sq_regression`. 
    1. In one cell, print the answer to verify that your parallel program is correct.
    2. In another cell time the computation. What is the speedup at `n_jobs=4`?
    3. Estimate the optimized fraction of the code ($p$) for this computation. Show your work.

In [13]:
# TODO code for 1A
from joblib import Parallel, delayed
def r_sq_regression_parallel (combo):
    Xp = X[:,combo]
    model = LinearRegression()
    model = model.fit(Xp,Y)
    return (combo,model.score(Xp,Y))

r_sq_list = Parallel(n_jobs=4)(delayed(r_sq_regression_parallel)(combo) for combo in combos)
r_sq_arr = np.array(r_sq_list, dtype=object)
r_sq_idx = np.argmax(r_sq_arr[:,1])
print(r_sq_arr[r_sq_idx])

[(2, 6, 14) 0.6095149101819038]


In [14]:
%%timeit 
# TODO code for 1B 
r_sq_list = Parallel(n_jobs=4)(delayed(r_sq_regression_parallel)(combo) for combo in combos)
r_sq_arr = np.array(r_sq_list, dtype=object)
r_sq_idx = np.argmax(r_sq_arr[:,1])

1.26 s ± 47.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**TODO Answer for 1B and 1C**

1B. The speedup is $\frac{1.62}{1.26} = 1.29$

1C. From Amdahl's Law, we can calculate Amdahl's number as $\frac{c}{c-1}\cdot \left(1-\frac{1}{s}\right)$ where $c$ is the number of cores and $s$ is the speedup. This is also listed in https://github.com/randalburns/pcds.2023/blob/main/ebook/concepts/01_strong_scaling.ipynb on the section Inferring parallelism from Amdahl's Law. Plugging in our numbers of $c=4$ and $s=1.29$, we get $\frac{4}{3}\cdot \left(1-\frac{1}{1.29}\right) = 0.30$

2. Use the batch size parameter to vary the number of tasks in each batch from 1,2,...128 @ n_jobs=4. You will need to look at the joblib documentation to read up about batch size.
   1. Plot your results (use %timeit -o to capture output)
   2. Model the problem as having two performance components: a fixed startup cost per batch ($C_B$) and perfect parallelism.  Estimate the batch startup cost.

In [15]:
# code for 2A
def run_with_batch_size(batch_size):
    r_sq_list = Parallel(n_jobs=4, batch_size=batch_size)(delayed(r_sq_regression_parallel)(combo) for combo in combos)
    r_sq_arr = np.array(r_sq_list, dtype=object)
    r_sq_idx = np.argmax(r_sq_arr[:,1])


In [16]:
time_results = []
batch_sizes = [1,2,4,8,16,32,64,128]
for batch_size in batch_sizes:
    result = %timeit -o run_with_batch_size(batch_size)
    time_results.append(result)   

3.27 s ± 306 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.2 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.85 s ± 60.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.53 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.32 s ± 59.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.26 s ± 80.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.07 s ± 49.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.11 s ± 78.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%matplotlib inline
plt.xlabel('batch sizes')
plt.ylabel('time (seconds)')
avg_results = [result.average for result in time_results]
plt.plot(batch_sizes, avg_results)
# TODO plot for 2A

In [18]:
# code for 2B
# Total time = execution time of tasks in parallel + C_b * number of batches created
# Slope of regression = C_b
# Number of batches = number of combos / batch_size
from sklearn.linear_model import LinearRegression
num_batches = [np.ceil(len(combos)/batch_size) for batch_size in batch_sizes]
model = LinearRegression()
model.fit(np.array(num_batches).reshape((-1,1)), np.array(avg_results))
print(model.coef_)



[0.00215341]


**TODO Answer for 2B**

Taking the slope of the line of our linear regression, we find that $C_b$ is about 2.15 milliseconds

3. Run the job with `prefer='threads'` and `prefer='processes'`. You do not need to vary batch size for this part. Which is more efficient?  Why? Consider our discussion of parallel threads in python.

In [19]:
%%timeit
r_sq_list = Parallel(n_jobs=4, batch_size=batch_size, prefer='threads')(delayed(r_sq_regression_parallel)(combo) for combo in combos)
r_sq_arr = np.array(r_sq_list, dtype=object)
r_sq_idx = np.argmax(r_sq_arr[:,1])

11 s ± 1.24 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
r_sq_list = Parallel(n_jobs=4, batch_size=batch_size, prefer='processes')(delayed(r_sq_regression_parallel)(combo) for combo in combos)
r_sq_arr = np.array(r_sq_list, dtype=object)
r_sq_idx = np.argmax(r_sq_arr[:,1])

1.2 s ± 51.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**TODO Answer to 3**

The processes version is more efficient. This is due to Python having a global interpreter lock. So each thread is concurrent, but not running in parallel as only one thread can acquire the lock at a time. On the other hand, multiple processes running multiple Python instances will not be limited by the global interpreter lock .